In [32]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import folium
from folium.plugins import HeatMap

In [23]:
crimes = pd.read_csv('datasets//Chicago_Crimes.csv')

In [24]:
crimes

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,13439321,JH237424,04/14/2024 12:00:00 AM,040XX S PRAIRIE AVE,0890,THEFT,FROM BUILDING,APARTMENT,False,False,...,3,38.0,06,1178707.0,1878256.0,2024,12/21/2024 03:40:46 PM,41.821236,-87.619921,"(41.821236024, -87.619920712)"
1,13437420,JH234779,04/14/2024 12:00:00 AM,023XX W CERMAK RD,2825,OTHER OFFENSE,HARASSMENT BY TELEPHONE,COMMERCIAL / BUSINESS OFFICE,False,False,...,25,31.0,26,1161210.0,1889347.0,2024,12/21/2024 03:40:46 PM,41.852052,-87.683801,"(41.852051675, -87.683800849)"
2,13428676,JH224478,04/14/2024 12:00:00 AM,043XX W LE MOYNE ST,0917,MOTOR VEHICLE THEFT,"CYCLE, SCOOTER, BIKE WITH VIN",STREET,False,False,...,36,23.0,07,1146960.0,1909501.0,2024,12/21/2024 03:40:46 PM,41.907640,-87.735587,"(41.907640473, -87.735587478)"
3,13429357,JH225293,04/14/2024 12:00:00 AM,039XX W ADAMS ST,143A,WEAPONS VIOLATION,UNLAWFUL POSSESSION - HANDGUN,STREET,True,False,...,28,26.0,15,1150158.0,1898721.0,2024,12/21/2024 03:40:46 PM,41.877997,-87.724121,"(41.877997275, -87.724120826)"
4,13430098,JH226395,04/14/2024 12:00:00 AM,011XX W 112TH PL,0890,THEFT,FROM BUILDING,RESIDENCE,False,False,...,21,75.0,06,1170856.0,1830157.0,2024,12/21/2024 03:40:46 PM,41.689421,-87.650123,"(41.6894214, -87.650123247)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249118,13805239,JJ217509,04/12/2025 12:00:00 AM,029XX W LOGAN BLVD,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,APARTMENT,False,False,...,1,22.0,26,1156478.0,1917149.0,2025,04/19/2025 03:41:24 PM,41.928440,-87.700416,"(41.928439867, -87.700415972)"
249119,13804023,JJ215813,04/12/2025 12:00:00 AM,094XX S HARVARD AVE,0430,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,False,False,...,9,49.0,04B,1175694.0,1842631.0,2025,04/19/2025 03:41:24 PM,41.723545,-87.632040,"(41.723545182, -87.632039508)"
249120,13803926,JJ215943,04/12/2025 12:00:00 AM,084XX S VINCENNES AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,...,21,71.0,08B,1173850.0,1848976.0,2025,04/19/2025 03:41:24 PM,41.740998,-87.638606,"(41.74099774, -87.638606337)"
249121,13803475,JJ215338,04/12/2025 12:00:00 AM,050XX S ABERDEEN ST,0530,ASSAULT,AGGRAVATED - OTHER DANGEROUS WEAPON,STREET,True,False,...,20,61.0,04A,1169838.0,1871348.0,2025,04/19/2025 03:41:24 PM,41.802477,-87.652657,"(41.802477219, -87.652657244)"


In [25]:
crimes['Date'] = pd.to_datetime(crimes['Date'], dayfirst = True, errors = 'coerce')

In [26]:
crimes['Year'] = crimes['Date'].dt.year

<h1> MOTORCYCLE THEFT </h1>
<h3> 2004 - 2025 </h3>
<hr>

In [27]:
crimes['Primary Type'].unique()

array(['THEFT', 'OTHER OFFENSE', 'MOTOR VEHICLE THEFT',
       'WEAPONS VIOLATION', 'BATTERY', 'ASSAULT',
       'CRIMINAL SEXUAL ASSAULT', 'CRIMINAL TRESPASS', 'CRIMINAL DAMAGE',
       'DECEPTIVE PRACTICE', 'SEX OFFENSE', 'ROBBERY', 'NARCOTICS',
       'HOMICIDE', 'INTERFERENCE WITH PUBLIC OFFICER', 'BURGLARY',
       'ARSON', 'OFFENSE INVOLVING CHILDREN', 'INTIMIDATION',
       'PUBLIC PEACE VIOLATION', 'CONCEALED CARRY LICENSE VIOLATION',
       'KIDNAPPING', 'STALKING', 'LIQUOR LAW VIOLATION', 'PROSTITUTION',
       'GAMBLING', 'OBSCENITY', 'PUBLIC INDECENCY', 'HUMAN TRAFFICKING',
       'OTHER NARCOTIC VIOLATION', 'NON-CRIMINAL'], dtype=object)

In [28]:
m_theft = crimes[crimes['Primary Type'] == 'MOTOR VEHICLE THEFT']

In [29]:
m_theft.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [30]:
import folium
from folium.plugins import HeatMap
# Aggregate incidents by location
aggregated_df = m_theft.groupby(['Latitude', 'Longitude']).size().reset_index(name='incident_count')

# # Normalize weights (optional)
aggregated_df['normalized_weight'] = (aggregated_df['incident_count'] - aggregated_df['incident_count'].min()) / \
                                     (aggregated_df['incident_count'].max() - aggregated_df['incident_count'].min())

# # Prepare heat_data
heat_data = aggregated_df[['Latitude', 'Longitude', 'normalized_weight']].values.tolist()

# # Create a base map
m = folium.Map(location=[30, 40], zoom_start=4)

# # Add heatmap layer
HeatMap(heat_data).add_to(m)

# # Save or display the map
m.save('terrorism_heatmap.html')
m

In [31]:
aggregated_df

,Latitude,Longitude,incident_count,normalized_weight
0,41.644590,-87.615880,2,0.025641
1,41.644782,-87.541164,1,0.000000
2,41.645732,-87.542389,1,0.000000
3,41.645796,-87.542472,5,0.102564
4,41.646066,-87.616354,1,0.000000
...,...,...,...,...
17335,42.022257,-87.669466,1,0.000000
17336,42.022521,-87.671488,1,0.000000
17337,42.022525,-87.672272,1,0.000000
17338,42.022526,-87.672401,1,0.000000


<h1> ANALYSIS 1  Weapon Vilation In  Streets</h1>

In [33]:
crimes.columns

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')

In [39]:
crimes['Primary Type']

<bound method Series.unique of 0                       THEFT
1               OTHER OFFENSE
2         MOTOR VEHICLE THEFT
3           WEAPONS VIOLATION
4                       THEFT
                 ...         
249118          OTHER OFFENSE
249119                BATTERY
249120                BATTERY
249121                ASSAULT
249122                  THEFT
Name: Primary Type, Length: 249123, dtype: object>

In [35]:
crimes['Location Description']

0                            APARTMENT
1         COMMERCIAL / BUSINESS OFFICE
2                               STREET
3                               STREET
4                            RESIDENCE
                      ...             
249118                       APARTMENT
249119                          STREET
249120                       APARTMENT
249121                          STREET
249122                          STREET
Name: Location Description, Length: 249123, dtype: object

In [36]:
# FILTERING
filter1 = crimes[crimes['Primary Type'] == 'WEAPONS VIOLATION']
filter2 = filter1[filter1['Year'] == 2024]
filter3 = filter2[filter2['Location Description'] == 'STREET']

In [37]:
street_violation = filter3.groupby(['Longitude', 'Latitude']).size().reset_index(name='Weapon Vilation In street')

In [47]:
# VISUALIZATION
heat_df = street_violation[['Latitude', 'Longitude', 'Weapon Vilation In street']].values.tolist()
map1 = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
HeatMap(heat_df).add_to(map2)

map1

<h1>ANALYSIS 2 Assault in appartment 2025</h1>

In [40]:
# FILTERING
filter1 = crimes[crimes['Primary Type'] == 'ASSAULT']
filter2 = filter1[filter1['Location Description'].str.contains('APARTMENT', na=False)]
filter3 = filter2[filter2['Year'] == 2025]

In [42]:
app_assault = filter3.groupby(['Longitude', 'Latitude']).size().reset_index(name='Appartment Assault')

In [49]:
# VISUALIZATION
heat_df = app_assault[['Latitude', 'Longitude', 'Appartment Assault']].values.tolist()
map2 = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
HeatMap(heat_df).add_to(map4)

map2

<h1>ANALYSIS 3  Theft In Streets</h1>

In [44]:
# FILTERING
filter1 = crimes[crimes['Primary Type'] == 'THEFT']
filter2 = filter1[filter1['Location Description'] == 'STREET']
filter3 = filter2[filter2['Year'] == 2025]

In [45]:
st_theft = filter3.groupby(['Longitude', 'Latitude']).size().reset_index(name='Street Theft')

In [50]:
# VISUALIZATION
heat_df = st_theft[['Latitude', 'Longitude', 'Street Theft']].values.tolist()
map3 = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
HeatMap(heat_df).add_to(map5)

map3